<img src="rc_logo.png">

#USGS Data Conversion and Cleaning#
<hr>


##Converting a CSV text file to a NetCDF file in python.##
Wednesday 24th of June 2015
<hr>
<ol>
<li>[Import the modules](#import)</li>
<li>[Read the CVS](#csv)</li>
<li>[Inspect the data](#inspect)</li>
<li>[Create a NetCDF file](#netcdf)</li>
<li>[Add metadata](#meta)</li>
<li>[Create the variables](#vars)</li>
<li>[Write the data](#data)</li>
<li>[Close the file](#close)</li>
</ol>

<a id='import'></a>

##Import all the needed modules

In [ ]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import datetime as dt

<a id='csv'></a>

##Read in the CVS file using pandas

In [ ]:
data = pd.read_csv('Walruses.csv', parse_dates=True, thousands=',')

In [ ]:
epoch = dt.datetime(1970,1,1)
data['time'] = [ t.total_seconds() for t in (pd.to_datetime(data['DateTimeUTC']) - pd.to_datetime(epoch))]
print(data['time'][0:2])

<a id='inspect'></a>

##Quickly inspect the data for correctness

In [ ]:
longLat = data[['Walrus', 'Longitude', 'Latitude']]
longLat[2:10:2]

In [ ]:
walrus_271 = data[data.Walrus == 271]
walrus_281 = data[data.Walrus == 281]
walrus_322 = data[data.Walrus == 322]
print(walrus_271.head(4))

<a id='netcdf'></a>

##Create a NetCDF

In [ ]:
ncf = nc.Dataset('Walrus_271.nc', mode='w')
print(ncf)

<a id='meta'></a>

##Create the metadata, attributes and conventions

In [ ]:
ncf.title        = ''
ncf.history      = ''
ncf.comment      = ''
ncf.source       = ''
ncf.references   = ''
ncf.Conventions  = "CF-1.6"
ncf.date_created = dt.datetime.utcnow().strftime('%d/%m/%Y %H:%M:%S.%f')
ncf.institution  = 'USGS'

In [ ]:
ncf.createDimension('time', None)
ncf.createDimension('longitude', len(walrus_271[['Longitude']]))
ncf.createDimension('latitude', len(walrus_271[['Latitude']]))

<a id="vars"></a>

##Create the variables

In [ ]:
time = ncf.createVariable('time', 'f8', ('time',), zlib=True)
time.units = 'seconds since 1970-01-01 00:00:00 UTC'
time.standard_name = 'time'
time.long_name = 'Epoch Time'

In [ ]:
lon = ncf.createVariable('longitude', 'f8', ('longitude',), zlib=True, least_significant_digit=8)
lon.units = 'degrees_east'
lon.standard_name = 'longitude'
lon.long_name = 'Longitude'

In [ ]:
lat = ncf.createVariable('latitude', 'f8', ('latitude',), zlib=True, least_significant_digit=8)
lat.units = 'degrees_north'
lat.standard_name = 'latitude'
lat.long_name = 'Latitude'

In [ ]:
w271 = ncf.createVariable('walrus_271', 'f8', ('time', 'latitude', 'longitude'), 
                        zlib=True, least_significant_digit=8)

<a id='data'><a>

##Write the data to the NetCDF file

In [ ]:
w271 = ncf.createVariable('walrus_271', 'f8', ('time', 'latitude', 'longitude'), 
                             zlib=True, least_significant_digit=8)
w271.id = 271
w271.standard_name = 'walrus_271'
w271.long_name = 'Rufus the Walrus, 271'

In [ ]:
ncf.variables['time'][:] = walrus_271['time'].values
ncf.variables['longitude'][:] = walrus_271['Longitude'].values
ncf.variables['latitude'][:] = walrus_271['Latitude'].values

<a id='close'></a>

##Close the NetCDF

In [ ]:
ncf.close()